In [0]:
# Defining the paths for different data

types = ["artists", "albums", "tracks"]

bronze_paths = {value: f"s3://my-raw-spotify-data/bronze/{value}/" for value in types} 

silver_paths = {value: f"s3://my-spotify-delta-lakehouse/silver{value}/" for value in types} 



In [0]:
# Reading unique ids from silver user recent played

played_unique_ids = spark.sql("""
SELECT DISTINCT album_id FROM my_spotify.silver.user_recent_played
""").collect()

played_unique_ids = [value.album_id for value in played_unique_ids]


In [0]:
#Excluding the ids we already have in the silver table to make api call

from delta.tables import DeltaTable

dim_table = DeltaTable.forPath(spark, silver_paths["albums"])


#Checking if the album already exists in the silver table
if dim_table.isDeltaTable(spark, silver_paths["albums"]):
    exist_ids = dim_table.select("album_id").collect()
    exist_ids = [value.album_id for value in exist_ids]
else:
    exist_ids = []

In [0]:
# Refreshing Token



import base64
import requests

# I use parameters instead of secrets as env variable because i am in Databricks Free Edition that has sevral limtiaitons

SPOTIFY_CLIENT_ID = dbutils.widgets.get("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = dbutils.widgets.get("SPOTIFY_CLIENT_SECRET")
SPOTIFY_REFRESH_TOKEN = dbutils.widgets.get("SPOTIFY_REFRESH_TOKEN")
def get_spotify_token():
    """
    Get access token from Spotify using refresh token.
    
    Returns:
        str: Access token if successful
        dict: Error response if failed
    """
    auth_str = f"{SPOTIFY_CLIENT_ID}:{SPOTIFY_CLIENT_SECRET}"
    b64_auth = base64.b64encode(auth_str.encode()).decode()

    try:
        resp = requests.post(
            "https://accounts.spotify.com/api/token",
            headers={"Authorization": f"Basic {b64_auth}"},
            data={
                "grant_type": "refresh_token",
                "refresh_token": SPOTIFY_REFRESH_TOKEN
            },
            timeout=10
        )

        if resp.status_code == 200:
            auth_data = resp.json()
            print("Token Generated")
            return auth_data['access_token']
        else:
            print({'statusCode': resp.status_code, 'body': resp.text})
            return {'statusCode': resp.status_code, 'body': resp.text}
            
    except requests.RequestException as e:
        return {'statusCode': 500, 'body': f'Request failed: {str(e)}'}
    

fresh_token = get_spotify_token()